Pour ce nouveau TP, nous allons poursuivre  l'utilisation des Transformers pour le traitement du langage naturel à travers la tâche devenue emblématique du domaine : la **génération de texte**.

Pour cela nous allons utiliser une base de données de tweets écrits par Donald Trump sur son compte twitter (@realdonaldtrump) entre 2009 et 2020.

<center><img src="https://www.alternatives-economiques.fr/sites/default/files/public/styles/16x9/public/field/image/000_36tp6xn.webp?orig=jpg&itok=bKKkrbct" width=600> </center>

A l'issue de l'entraînement, on obtiendra un modèle permettant de générer, par exemple, les phrases suivantes :

```I love Putin. He is a great guy. But I think he is a weak man. I see him as a very tough guy. He is not very smart, but he is very strong. He has a lot of things to say.``` (obtenu à partir d'une phrase de départ 'I love Putin')

```Obama is  the most corrupt, the most corrupt President ever. As a private citizen I have no interest in paying him any money. He is a fraud!``` (obtenu à partir de la phrase de départ 'Obama is')

**NB:** Vous allez voir que la quantité de code à compléter dans ce TP est relativement faible. Si vous terminez en avance, n'hésitez pas à réfléchir à comment adapter ce code à votre projet !




# Présentation des données

Commencez par télécharger les données :

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
from keras import models
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

# Téléchargement des données
path_to_file = tf.keras.utils.get_file('realdonaltrump.csv', 'https://drive.google.com/uc?export=download&id=1s1isv9TQjGiEr2gG__8bOdBFvQlmepRt')

# Lecture du CSV dans une liste de tweets et concaténation des tweets dans la variable text
tweets = []
text = ''
with open(path_to_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        tweets.append(row['content'])
        text += row['content']


# Affichage des 10 premiers tweets
print(tweets[:10])

Le fichier CSV compte 43339 tweets, accompagnés d'autres métadonnées que nous n'utiliserons pas.



In [ ]:
# Nombre total de caractères du dataset
print(f'Longueur totale du texte: {len(text)} caractères')

In [ ]:
# Extraction des caractères uniques du texte
vocab = sorted(set(text))
print(f'{len(vocab)} unique caractères')

In [ ]:
# Calcul des fréquences d'apparition de chaque caractère
char_freq = {}
for char in text:
  if char not in char_freq:
    char_freq[char] = 1
  else:
    char_freq[char] += 1


# Affichage du vocabulaire trié par ordre de fréquence décroissant en filtrant les caractères apparaissant moins de 100 fois
for char in sorted(char_freq, key=lambda x: char_freq[x], reverse=True):
  print(f'{char}: {char_freq[char]}')



Nous avons donc 147 caractères dans la base de données, dont certains sont extrêmement rares. Il aurait certainement été possible de remplacer les caractères rares par un token particulier (\<unk\>, pour *unknown*) mais ils sont suffisamment rares pour ne pas causer de problèmes particuliers.

# Entraînement complet d'un modèle relativement petit

## Tokenization et formatage des données

Pour plus de simplicité dans cette partie, nous allons adopter une *Tokenization* basée sur les caractères. Chaque caractère se verra assigner un identifiant unique, et nous allons ajouter deux tokens supplémentaires, l'un pour désigner le début d'un tweet (\<sot\>, *start of tweet*), l'autre pour en désigner la fin (\<eot\>).

Nous allons créer deux structures de données permettant de passer d'une représentation caractère vers les indices des tokens (```char_to_ind```) et vice versa (```ind_to_char```).

In [ ]:
# Création du dictionnaire chatr_to_ind pour associer un indice unique à chaque caractère, en commençant à l'indice 2
char_to_ind = {char: idx+2 for idx, char in enumerate(vocab)}
char_to_ind['<sot>'] = 0 # Start of tweet
char_to_ind['<eot>'] = 1 # End of tweet

# Création du dictionnaire ind_to_char pour associer à chaque indice un caractère
ind_to_char = {idx: char for char, idx in char_to_ind.items()}


print(char_to_ind)
print(ind_to_char)

Nous pouvons maintenant passer à la Tokenization de la base de données, c'est-à-dire à la conversion de chaque tweet en une liste d'identifiants représentant le tweet. Par exemple, pour le premier tweet :

In [ ]:
print(tweets[0])

tokenized_tweet_0 = [char_to_ind['<sot>']]
for char in tweets[0]:
  tokenized_tweet_0.append(char_to_ind[char])
tokenized_tweet_0.append(char_to_ind['<eot>'])

print(tokenized_tweet_0)


Notez que nous avons transcrit le tweet en rajoutant \<sot\> au début de la liste et \<eot\> à la fin.

In [ ]:
# Parcours de la liste de tweets et création d'une liste de listes d'indices des caractères composant chaque tweet
tweets_ind = []
for tweet in tqdm(tweets):
  tweet_ind = [char_to_ind['<sot>']]
  for char in tweet:
    tweet_ind.append(char_to_ind[char])
  tweet_ind.append(char_to_ind['<eot>'])
  tweets_ind.append(tweet_ind)


Il est intéressant de visualiser l'ordre de grandeur des tweets une fois tokenizés :

In [ ]:
# Affichage d'un histogramme des longueurs des tweets tokenizés
tweet_lengths = [len(tweet) for tweet in tweets_ind]
plt.hist(tweet_lengths, bins=100)
plt.title('Histogramme des longueurs des tweets')
plt.xlabel('Longueur')
plt.show()

La grande majorité des tweets ont moins de 150 caractères. Ceci permettra de choisir la taille du contexte un peu plus tard.


Nous allons maintenant formater les données pour l'apprentissage. Notre modèle de langage sera un modèle *Many-to-many* qui prédira pour chaque token de la séquence le token suivant. Souvenez-vous, nous avons fait la même chose l'an passé en TP sur les RNN (à l'époque, avec des noms de famille).

<center><img src="https://drive.google.com/uc?id=1TfgprY0yB4blIlRbHHr3S8UQB3nid5zo" width=600> </center>
<caption><center> Génération de nom à l'aide d'une cellule récurrente</center></caption>

Nous devons donc créer des matrices X et Y contenant des séquences de tweets, où les séquences de Y sont décalées d'un cran vers la droite.

Pour cela, nous allons commencer par concaténer l'intégralité des tweets tokenizés du dataset :

In [ ]:
# Concaténation de tous les tweets tokenizés dans une seule liste
all_tweets_ind = []
for tweet in tweets_ind:
  all_tweets_ind += tweet

print(len(all_tweets_ind))

Nous allons maintenant fixer une taille de contexte. Ceci est un hyperparamètre que vous pourrez ajuster un peu plus tard, mais **gardez à l'esprit que les couches d'attention ont une complexité qui croit avec le carré de la taille du contexte**. Autrement dit, plus le contexte est petit, plus votre modèle sera rapide. Mais, bien sûr, moins il conservera de mémoire pour générer la fin d'un tweet à partir du début.

In [ ]:
MAX_LEN = 50

Pour créer les variables X et Y, nous allons découper l'intégralité de notre dataset en morceaux de longueur ```MAX_LEN```+1. Chaque morceau sera ensuite réparti entre la variable X (les ```MAX_LEN``` premiers tokens) et la variable Y (les ```MAX_LEN``` derniers tokens).

In [ ]:
import numpy as np
import math

# Création des variables X et Y
# On découpe l'ensemble des tweets concaténés en MAXLEN+1 caractères consécutifs : les MAXLEN premiers sont les x et les MAXLEN derniers sont les y
### A COMPLETER


print(X, Y)

**Résultat attendu** :
```
[[ 0. 36. 70. ... 66. 85. 70.]
 [48. 74. 72. ... 80. 81.  2.]
 [70. 79.  2. ... 72.  2. 80.]
 ...
 [70.  2. 85. ... 90.  2. 85.]
 [79. 74. 72. ... 84. 81. 67.]
 [26. 58.  1. ... 17. 56. 46.]] [[36. 70.  2. ... 85. 70.  2.]
 [74. 72. 73. ... 81.  2. 54.]
 [79.  2. 46. ...  2. 80. 79.]
 ...
 [ 2. 85. 80. ...  2. 85. 80.]
 [74. 72. 73. ... 81. 67. 55.]
 [58.  1.  0. ... 56. 46. 77.]]
 ```

**NB** : peut-être êtes vous choqué·e par le fait que certaines données de notre ensemble d'apprentissage se retrouvent ainsi "à cheval" sur deux tweets consécutifs, qui n'ont potentiellement rien à voir.

Cela nous permet d'éviter l'écueil de devoir gérer les différences de longueur des tweets. Si nous avions choisi une taille de contexte de, disons, 300 (autour de la longueur maximum des tweets), nous aurions dû compléter la grande majorité des tweets avec un token spécial (pour faire du *padding*, \<pad\>). Ce token aurait alors été en écrasante majorité dans la base de données, et le modèle aurait eu plus de mal à apprendre à cause du déséquilibre de classes induit par cette solution.

## Création du modèle

Pour cette étape, je vous renvoie aux explications du TP4 de la semaine passée car vous allez pouvoir réutiliser le code écrit à cette occasion.

Du point de vue de l'implémentation, une unique chose sépare un réseau de type **GPT** (modèle décodeur, que nous implémentons cette semaine) d'un réseau de type **BERT** (modèle encodeur, implémenté au TP4) : la couche d'auto-attention.

Dans **BERT**, qui est **B**idirectionnel, l'attention d'un token peut être portée à tous les tokens de la séquence, qu'ils soient situés avant ou après dans l'ordre de la séquence.

Dans **GPT**, qui sera utilisé pour faire de la **G**énération, il est strictement interdit de porter attention à des tokens qui sont postérieurs au token courant ! En effet, comme l'objectif de **GPT** est de générer progressivement du texte, cela n'aurait aucun sens de porter attention à des tokens que nous allons générer dans le futur.

A vous de lire attentivement la documentation de la classe [MultiHeadAttention](https://keras.io/api/layers/attention_layers/multi_head_attention/) pour déterminer quel paramètre l'on doit positionner poru masquer la couche d'attention.

In [ ]:
class TransformerBlock(layers.Layer):
    # embed_dim désigne la dimension des embeddings maintenus à travers les différentes couches,
    # et num_heads le nombre de têtes de la couche d'attention.
    # DANS CETTE FONCTION, ON NE FAIT QUE DEFINIR LES COUCHES
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        # Définition des différentes couches qui composent le bloc
        # Couche d'attention
        self.att =
        # Première couche de Layer Normalization
        self.layernorm1 =
        # Couche Dense (Feed-Forward)
        self.ffn =
        # Deuxième couche de normalisation
        self.layernorm2 =

    # DANS CETTE FONCTION, ON APPELLE EXPLICITEMENT LES COUCHES DEFINIES DANS __init__
    # ON PROPAGE DONC LES ENTREES inputs A TRAVERS LES DIFFERENTES COUCHES POUR OBTENIR
    # LA SORTIE
    def call(self, inputs):
        # Application des couches successives aux entrées
        return ...

La partie Embedding est inchangée :

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # Définition des différentes couches qui composent le bloc Embedding
        # Embedding de mot
        self.token_emb =
        # Embedding de position
        self.pos_emb =

    def call(self, x):
        # Calcul de l'embedding à partir de l'entrée x
        # ATTENTION : UTILISER UNIQUEMENT DES FONCTIONS TF POUR CETTE PARTIE
        # Récupération de la longueur de la séquence
        maxlen =
        # Création d'un vecteur [0, 1, ..., maxlen] des positions associées aux
        # mots de la séquence (fonction tf.range)
        positions =
        # Calcul des embeddings de position
        positions_emb = ...
        # Calcul des embeddings de mot
        words_emb = ...
        return

Enfin, vous devez assembler un réseau qui ressemble à GPT (ci-dessous) :    


<center><img src="https://drive.google.com/uc?id=1w1CyLROPq-EWMd-Spr6wR596QEx1KpNa" width=200> </center>
<caption><center>  Schéma de l'architecture de GPT</center></caption>

Je vous propose des valeurs pour EMBED_DIM et NUM_HEADS, mais vous êtes bien sûr libres de les modifier.

Pour obtenir des résultats similaires aux miens, vous devez utiliser 3 blocs transformers, et terminer le réseaux avec une couche dense de 100 neurones, et enfin la couche de sortie (à vous de déterminer quoi y mettre !)


In [ ]:
EMBED_DIM = 128  # Dimension de l'embedding pour chaque mot
NUM_HEADS = 8  # Nombre de têtes d'attention
VOCAB_SIZE = len(char_to_ind)
### A COMPLETER
inputs = keras.Input(shape=(MAX_LEN,))
embedding_layer = ...
...
outputs = keras.layers.Dense(...

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Il reste à entraîner le modèle. Pour faire en sorte que ça ne soit pas trop long, je n'ai mis que 5 epochs, mais des entrainements plus longs conduisent à de meilleurs résultats.

Grosse difficulté de cette partie : quelle fonction de coût devez-vous utiliser ? Un petit indice, vous la trouverez sur [cette page](https://keras.io/api/losses/) au rayon *Probabilistic losses*.

In [ ]:
### A COMPLETER
model.compile(
    optimizer="adam", loss="...", metrics=["accuracy"]
)
history = model.fit(
    X, Y, batch_size=16, epochs=5)

## Génération d'un résultat

Profitez du temps d'entrainement de la cellule précédente pour prendre un café et lire le code permettant d'implémenter la génération. Il n'y a rien à compléter mais ce code n'est pas si facile à comprendre.

In [ ]:
# Début de séquence : mettre '' si l'on veut générer un nom de zéro
seed_seq = 'Obama is'

i = 0

# Création de la séquence qui va être fournie en entrée du réseau :
# On ajoute un token <sos> au démarrage, et on transcrit en la séquence d'id correspondante
input_seq = [char_to_ind['<sot>']]
for s in seed_seq:
  input_seq.append(char_to_ind[s])
  i+=1

last_char = -1


# On génère des séquences de taille inférieure à MAX_LEN, et on s'arrête lorsque
# l'on génère un token <eot> (id 1)
while last_char != 1 and i < 200:
  # La séquence d'entrée doit être de dimension BATCH_SIZE x MAX_LEN
  # soit en fait ici 1 x MAX_LEN
  input = np.zeros((MAX_LEN))
  if i < MAX_LEN:
    input[:i+1] = np.array(input_seq)
  else:
    input = np.array(input_seq[-MAX_LEN:])

  input = np.expand_dims(input, 0)

  # Prédiction du modèle sur la séquence en cours
  pred = model.predict(input, verbose=0)
  # En sortie, on a 1 x MAX_LEN x 149

  # Échantillonnage du caractère généré à partir de la distribution de probabilité
  # prédite par le modèle pour le dernier élément de la séquence
  last_char = np.random.choice(VOCAB_SIZE, 1, p=pred[0, min(i, MAX_LEN-1)])[0]

  # Ajout du caractère à la séquence générée
  input_seq.append(last_char)
  i += 1

# Affichage du nom généré
generated_tweet = ''
for s in input_seq:
  if s != 0 and s != 1:
    generated_tweet+=ind_to_char[s]

print(generated_tweet)

Voici quelques exemples de résultats que j'obtiens après 5 epochs (la loss est autour de 1,5, les résultats s'améliorent un peu à mesure qu'elle baisse) :     

```Obama is no manufacture, there's pruingistricty. Think IME DENDY DUNT NATED more to construction, the holden for working on foom reproud & thank you!pic.twitter.com/wNXxY03ETLbE```

```Obama is a lot of @ CNN and the This is country.```

```Obama is an oposet bad all times and hill out the world in me.We dected were progress on @ dmiasonstroning Bernie, VOTES and ar the crime night Donumer more,. Laput are . I create pla att to bluin Sco```

Les résultats peuvent paraître décevants, mais la plupart des mots générés existent : c'est déjà satisfaisant, car le modèle les génère caractère par caractère !

En augmentant la profondeur du réseau (nombre de blocs transformers), la dimension des *Embeddings* et bien sûr le nombre d'epochs, il est possible de faire mieux mais nous serons de toute façon limités par la Tokenization caractère qui rend le problème un peu plus difficile.

Il est plus intéressant d'explorer plutôt le *fine-tuning* d'un modèle de langage déjà pré-entraîné.



# Fine-tuning d'un petit LLM

Commencez par installer toutes les librairies suivantes. Cela prendra un peu de temps (2e café), et vous risquez de devoir redémarrer votre machine virtuelle.

In [ ]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install accelerate
!pip install bitsandbytes

Nous allons utiliser le modèle [OPT](https://arxiv.org/abs/2205.01068), pour *Open Pretrained Transformer* sorti par Meta en 2022. Il s'agit d'une collection de LLM de tailles allant de 125 millions de paramètres à 175 milliards de paramètres, pré-entraînés, et complètement libres d'utilisation.

Comme nous l'avons vu la semaine passée, chargeons le Tokenizer et le modèle associés :

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name, force_download=True)

Dans le bloc suivant, nous préparons la base de données en créant un objet Dataset à partir de nos tweets (non tokenizés), et en les passant ensuite dans le Tokenizer de opt-125m.

In [ ]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": tweets})
print(dataset)

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Pour accélérer l'entraînement, nous utilisons une méthode que je n'ai pas eu le temps de vous présenter en cours :    [LoRA](https://arxiv.org/abs/2106.09685), pour *Low Rank Approximation*.

L'idée de cette méthode est de modifier incrémentalement les poids synaptiques d'un réseau à l'aide d'une décomposition matricielle.

Ainsi une matrice $W \in \mathbb{R}^{d \times k}$ représentant les poids synaptiques d'une couche du réseau peut s'écrire
$$ W' = W + AB$$
où $A \in \mathbb{R}^{d \times r}$ et $B \in \mathbb{R}^{r \times k}$, avec $r$ un rang très petit par rapport aux dimensions $d$ et $k$.

Les deux matrices $A$ et $B$ n'ont que peu de paramètres, et leur produit $AB$ sera de rang très faible. La méthode LoRA, utilisée pour le fine-tuning de LLM, consiste à bloquer les paramètres de $W$ et à autoriser uniquement l'entraînement de $A$ et $B$. Malgré leur faible nombre de paramètres, le résultat permet de modifier significativement les prédictions du LLM tout en étant plus léger à entraîner, et moins sujet au sur-apprentissage.

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Nous pouvons maintenant lancer le *fine-tuning*

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    max_steps=1000,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="no",
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

Le bloc suivant permet de définir le processus de génération de nouveaux tweets à partir de notre modèle. Il est possible d'initialiser ces tweets avec un prompt :

In [ ]:
from transformers import pipeline

# Configurer un pipeline de génération
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Obama is "
results = generator(prompt, max_length=50, num_return_sequences=3, do_sample=True, temperature=0.7)

for i, result in enumerate(results):
    print(f"--- Génération {i+1} ---")
    print(result['generated_text'])

Si le code précédent est trop long à exécuter, vous pouvez charger les poids suivants, que j'ai moi-même fine-tunés pendant 3 epochs (~45 min sur Google Colab)

In [ ]:
!wget https://acarlier.fr/tp/fine_tuned_opt_125m.zip
!unzip fine_tuned_opt_125m.zip

# Path to the extracted model
model_path = './'

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
from transformers import pipeline

# Configurer un pipeline de génération
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Obama is "
results = generator(prompt, max_length=50, num_return_sequences=3, do_sample=True, temperature=0.7)

for i, result in enumerate(results):
    print(f"--- Génération {i+1} ---")
    print(result['generated_text'])